# Model-Agnostic Input Data Preprocessing in CONFLUENCE

## Introduction

This notebook focuses on the model-agnostic preprocessing steps for input data in CONFLUENCE. Model-agnostic preprocessing involves tasks that are common across different hydrological models, such as data acquisition, quality control, and initial formatting.

Key steps covered in this notebook include:

1. Acquiring meteorological and geospatial data for the study area
2. Performing data quality checks and gap-filling
3. Standardizing data formats and units
4. Spatial interpolation or resampling of data to match the model domain

In this preprocessing stage we ensure that our input data is consistent, complete, and properly formatted before we move on to model-specific preprocessing steps. By the end of this notebook, you will have a clean, standardized dataset ready for further model-specific processing.